In [68]:
'''In the output file "null" occures when there is no data point to convert, and nan coresponds to having no redshift
value to convert with. All lists named a,b,c,d,... are used as columns 0,1,2,3,... in the output file. The title of each collumn is commented.'''


# Import modules:

from uncertainties import ufloat #Documentation: https://pythonhosted.org/uncertainties/user_guide.html
from uncertainties.umath import *
from numpy import pi as pi
from numpy import nan as nan
import astropy
from astropy.cosmology import WMAP9 as cosmo
from astropy.table import Table, Column
asc = astropy.io.ascii

In [1]:
#This cell defines file paths and initial parameters:

#Input and output file information:
Redshift_file='Redshift.csv'
Red_name = 0 #column number of SN names from Redshift_file
Red_val = 4 #column number of redshift values from Redshift_file

Data_file='GALEX_nuv_fuv_flux_(UTF_8)T.csv'
Data_name = 0 #column number of SN names from Data_file
Data_val = 1 #column number of counts per second from Data_file
Data_exp = 2 #column number of exposure time from Data_file

Output_file="output.csv"

#Initial paramaters are defined
flux_conv = 2.06*10**(-16) #(erg cm-2 Å-1)/count
        #NUV: Flux = 2.06*10**(-16) x CPS, FUV: Flux = 1.40*10**(-15) x CPS  -- Reference: http://asd.gsfc.nasa.gov/archive/galex/FAQ/counts_background.html

Pixel_area=(1.5/60)**2 #Pixel width is given in arcseconds, converted to arcmin, and squared to find area in arcmin^2.

'''need to add real exposure time to data_file'''

'need to add real exposure time to data_file'

In [80]:
#Since we are using ufloats we create a function that will take list of ufloats and 
    #generate a list of principle values and a list of errors

def seperate(l,l_n,l_s):
    z=0
    while z<len(l):
        if l[z] != ('null' or 'nan'):
            l_n.append(";".join([str(v) for v in [x.n for x in c[z]]]))
            l_s.append(";".join([str(v) for v in [x.s for x in c[z]]]))
        else:
            l_n.append(c[z])
            l_s.append(c[z])
        z=z+1

In [81]:
#Input files are read into tables using ascii module:

data = Table(asc.read(Data_file))
red = Table(asc.read(Redshift_file)) 

In [82]:
#This cell uses the table "red" to create a dictionary of redshift values {SN name: Redshift value}
# and a dictionary of conversion factors {SN name: kpc/arcmin}

conv={}
redshift={}
n=0

#Create two dictionaries of the form {SNname: (redshift,error)}. Error in redshift is taken as (1)/1000
for row in red:
    conv[red[n][Red_name].replace(" ", "")]=ufloat(red[n][4],(2/1000))
    redshift[red[n][Red_name].replace(" ", "")]=ufloat(red[n][4],(2/1000))
    n=n+1
    
#Convert redshift values into conversion factors. "conv" is changed to the form {SNname: conversion factor}
for key, value in conv.items():
    conv[key] = ufloat(
        
                    ((1/cosmo.kpc_comoving_per_arcmin(value.n))**2).value,
        
                    abs(((1/cosmo.kpc_comoving_per_arcmin(value.n+value.s))**2).value
                    -((1/cosmo.kpc_comoving_per_arcmin(value.n-value.s))**2).value)/2
        
                      )
    
'''Need to correct formula for error in redshift'''

/Users/Daniel/anaconda/lib/python3.4/site-packages/numpy/ma/core.py:3900: UserWarning: Warning: converting a masked element to nan.
  warnings.warn("Warning: converting a masked element to nan.")
/Users/Daniel/anaconda/lib/python3.4/site-packages/scipy/integrate/quadpack.py:352: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  warnings.warn(msg, IntegrationWarning)


'Need to correct formula for error in redshift'

In [83]:
#This cell creates a list for the supernova names and a list for their corresponding pixel values

a=[] #SN name

b_n=[] #Counts per second value
b_s=[] #counts per second error
b=[] # combined counts per second list using ufloats


#Add appropriate values to lists a, b, and c:
for row in data: 
    a.append(row[Data_name])
    
    if row[Data_val]!='':
        #print(row[Data_val])
        b_n = row[Data_val].split(';')
        b_s = [(float(a)*float(b))**(1/2) for a,b in zip(row[Data_val].split(";"),row[Data_exp].split(";"))] #error in cps = sqrt(cps*eposure time)
        b.append([ufloat(float(a),float(b)) for a,b in zip(b_n,b_s)])

    else:
        b.append("null")


In [84]:
#This cell converts pixel values to flux values:

c=[] #Flux in ufloat form

for elt in b:
    if elt !='null': 
        l = [x*flux_conv for x in elt]
        c.append(l)
    else:
        c.append("null")
 
'''Error in cps -> flux conversion factor was not considered'''

'Error in cps -> flux conversion factor was not considered'

In [85]:
#This cell converts flux values to luminosity values

d=[] #luminosity in ufloat form

n=0
while n < len(c):
    if (a[n].replace(" ", "")) in conv.keys():
        if c[n]!="null":
            r_n = cosmo.comoving_distance(redshift[a[n].replace(" ", "")].n).cgs.value
            r_s=(
                abs((cosmo.comoving_distance(redshift[a[n].replace(" ", "")].n+redshift[a[n].replace(" ", "")].s).cgs)
                    -(cosmo.comoving_distance(redshift[a[n].replace(" ", "")].n-redshift[a[n].replace(" ", "")].s).cgs)/2).value          
                )
            r=ufloat(r_n,r_s)
            lum = [ x*4*pi*(r)**2 for x in c[n] ]
            d.append(lum)
        else:
            d.append("null")
    else:
        d.append([ufloat(nan,nan)])
    n=n+1
    
'''error in comoving_distance should be changed to a specific equation.'''

/Users/Daniel/anaconda/lib/python3.4/site-packages/scipy/integrate/quadpack.py:352: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  warnings.warn(msg, IntegrationWarning)


'error in comoving_distance should be changed to a specific equation.'

In [86]:
#This cell calculates the surface brightness and converts values to proper units.


e=[] #Surface Brightness (erg s-1 A-1 kpc^-2)

n=0
while n < len(d):
    if d[n]!="null":
        if (a[n].replace(" ", "")) in conv.keys():
            
            #Convert values in the list from px^-1 to arcmin^-2 and then from arcmin^-2 to kpc^-2
            arcmin = [ x/Pixel_area for x in d[n] ]
            kpc = [ x*(conv[(a[n].replace(" ", ""))].n) for x in arcmin ]
            e.append(kpc)
            
        else:
            e.append("nan")
    else:
        e.append("null")
    n=n+1
'''Error in pixel area and conversion factor were not considered'''

'Error in pixel area and conversion factor were not considered'

In [88]:
#We create lists for each measurment and error and write these lists to the output file

b_n, b_s, c_n, c_s, d_n, d_s, e_n, e_s=[], [], [], [], [], [], [], []

seperate(b,b_n,b_s)
seperate(c,c_n,c_s)
seperate(d,d_n,d_s)
seperate(e,e_n,e_s)

asc.write([a, b_n, b_s, c_n, c_s, d_n, d_s, e_n, e_s], Output_file, names=[
        'SN Names', 
        'Pixel Value N/s', 'Pixel Value Error N^(1/2)',
        'Flux (erg s-1 cm-2 A-1 px-1)','Flux Error (erg s-1 cm-2 A-1 px-1)',
        "Luminosity (erg s-1 A-1 px-1)", "Luminosity Error (erg s-1 A-1 px-1)",
        "Surface Brightness (erg s-1 A-1 kpc^-2)", "Surface Brightness Error (erg s-1 A-1 kpc^-2)" 
        ], delimiter=",")

#do I need to close the file?